In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

: 

In [18]:
import json
from datetime import datetime, timezone
import http
import requests

In [2]:
from ultralytics import YOLO

# Load model
model = YOLO("yolo11n.pt")

# Train on GPU
results = model.train(
    data=r"C:\Users\SOHAM\Downloads\Major_project-20250818T044734Z-1-001\Major_project\yoloo\data.yaml",
    epochs=100,
    imgsz=640,
    device=0   # 0 = first GPU
)


ModuleNotFoundError: No module named 'ultralytics'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print("CUDA available:", torch.cuda.is_available())


CUDA available: True


In [6]:
new_model = YOLO(r"C:\Users\SOHAM\Downloads\Major_project-20250818T044734Z-1-001\Major_project\runs\detect\train\weights\best.pt")

In [27]:
import cv2
im = cv2.imread(r'C:\Users\SOHAM\Downloads\Major_project-20250818T044734Z-1-001\Major_project\Testing_images\panth.jpg')
res = new_model.predict(im)



In [28]:
for re in res:
    boxes = re.boxes  # Boxes object for bounding box outputs
    masks = re.masks  # Masks object for segmentation masks outputs
    keypoints = re.keypoints  # Keypoints object for pose outputs
    probs = re.probs  # Probs object for classification outputs
    obb = re.obb  # Oriented boxes object for OBB outputs
    re.show()  # display to screen

In [ ]:
import cv2
import numpy as np

from collections import Counter
from ultralytics import YOLO

# --- settings ---
VIDEO = r"C:\Users\SOHAM\Downloads\Major_project-20250818T044734Z-1-001\Major_project\Testing_images\video_preview_h264.mp4"
WEIGHTS = r"C:\Users\SOHAM\Downloads\Major_project-20250818T044734Z-1-001\Major_project\runs\detect\train\weights\best.pt"             # your trained model
CONF_THRES = 0.5
IMG_SIZE = 640
TRACKER = "bytetrack.yaml"      # robust tracker
IOU_MERGE = 0.5                 # merge new track with an existing entity if IoU > this
DIST_FRAC = 0.15                # or if centroid distance < this * image diagonal
MAX_AGE_SEC = 1.0               # how long (sec) an entity may disappear and still be merged
CLASS_WHITELIST = None          # e.g. {"Panthera_leo"} to count only lions

# --- helpers ---
def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1, inter_y1 = max(ax1, bx1), max(ay1, by1)
    inter_x2, inter_y2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, inter_x2 - inter_x1), max(0, inter_y2 - inter_y1)
    inter = iw * ih
    if inter == 0:
        return 0.0
    area_a = (ax2 - ax1) * (ay2 - ay1)
    area_b = (bx2 - bx1) * (by2 - by1)
    return inter / float(area_a + area_b - inter + 1e-6)

def centroid(box):
    x1, y1, x2, y2 = box
    return (0.5*(x1+x2), 0.5*(y1+y2))

# --- entity store ---
class Entity:
    _next_id = 0
    def __init__(self, box, frame_idx, class_name, track_id=None):
        self.eid = Entity._next_id; Entity._next_id += 1
        self.box = box
        self.last_frame = frame_idx
        self.class_hist = Counter([class_name])
        self.canonical = class_name
        self.track_ids = set()
        if track_id is not None:
            self.track_ids.add(int(track_id))
    def update(self, box, frame_idx, class_name, track_id=None):
        self.box = box
        self.last_frame = frame_idx
        self.class_hist[class_name] += 1
        self.canonical = self.class_hist.most_common(1)[0][0]
        if track_id is not None:
            self.track_ids.add(int(track_id))

# --- main ---
model = YOLO(WEIGHTS)

cap = cv2.VideoCapture(VIDEO)
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
diag = (w**2 + h**2) ** 0.5
max_age_frames = int(MAX_AGE_SEC * fps)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("output_video.mp4", fourcc, fps, (w, h))

entities = []  # list[Entity]
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # Track with ByteTrack
    results = model.track(
        frame,
        imgsz=IMG_SIZE,
        conf=CONF_THRES,
        tracker=TRACKER,
        persist=True,
        verbose=False
    )

    annotated = results[0].plot()

    boxes = results[0].boxes
    if boxes is not None and len(boxes) > 0:
        xyxy = boxes.xyxy.cpu().numpy()
        clsi = boxes.cls.cpu().numpy().astype(int)
        names = [model.names[i] for i in clsi]
        tids = boxes.id.cpu().numpy().astype(int).tolist() if boxes.id is not None else [None]*len(xyxy)

        for box, name, tid in zip(xyxy, names, tids):
            if CLASS_WHITELIST and name not in CLASS_WHITELIST:
                continue

            # find best entity to merge with
            best_e, best_score = None, -1
            cx, cy = centroid(box)
            for e in entities:
                if frame_idx - e.last_frame > max_age_frames:
                    continue  # too old to match
                iou = iou_xyxy(box, e.box)
                ex, ey = centroid(e.box)
                dist_ok = ( ((cx-ex)**2 + (cy-ey)**2)**0.5 ) < (DIST_FRAC * diag )
                # score: prefer IoU, then distance
                score = iou + (0.5 if dist_ok else 0.0)
                if score > best_score and (iou >= IOU_MERGE or dist_ok):
                    best_e, best_score = e, score

            if best_e is None:
                # new entity
                entities.append(Entity(box, frame_idx, name, tid))
            else:
                best_e.update(box, frame_idx, name, tid)

    # prune very old entities (optional – keeps memory tidy)
    entities = [e for e in entities if frame_idx - e.last_frame <= 5*max_age_frames]

    # counts by canonical class
    counts = Counter()
    for e in entities:
        if CLASS_WHITELIST and e.canonical not in CLASS_WHITELIST:
            continue
        counts[e.canonical] += 1
    total_unique = sum(counts.values())

    # overlay
    y = 40
    cv2.putText(annotated, f"Total Unique Animals: {total_unique}", (20, y),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0,255,0), 4); y += 40
    for cls, c in counts.items():
        cv2.putText(annotated, f"{cls}: {c}", (20, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,255), 3); y += 32

    sendData(species, count, behaviour, latitude, longitude, camera_id)
    
    cv2.imshow("YOLO Video", annotated)
    out.write(annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# final report
final_counts = Counter()
for e in entities:
    if CLASS_WHITELIST and e.canonical not in CLASS_WHITELIST:
        continue
    final_counts[e.canonical] += 1
print("Final unique animals:", dict(final_counts))


Final unique animals: {'Panthera_leo': 2}


In [ ]:
import cv2
import numpy as np
import asyncio
import aiohttp
import threading

from collections import Counter
from ultralytics import YOLO

# --- settings ---
VIDEO = r"C:\Users\SOHAM\Downloads\Major_project-20250818T044734Z-1-001\Major_project\Testing_images\video_preview_h264.mp4"
WEIGHTS = r"C:\Users\SOHAM\Downloads\Major_project-20250818T044734Z-1-001\Major_project\runs\detect\train\weights\best.pt"
CONF_THRES = 0.5
IMG_SIZE = 640
TRACKER = "bytetrack.yaml"
IOU_MERGE = 0.5
DIST_FRAC = 0.15
MAX_AGE_SEC = 1.0
CLASS_WHITELIST = None

# --- API settings ---
API_ENDPOINT = 'http://127.0.0.1:8000/animals/' # Your API endpoint
SEND_INTERVAL_FRAMES = 30 # Send data every 30 frames

# --- NEW: Asynchronous data sending function ---
async def send_data(session, species, count, behaviour, latitude, longitude, camera_id):
    """Sends animal data to the API asynchronously."""
    payload = {
        'species': species,
        'count': count,
        'behaviour': behaviour,
        'latitude': latitude,
        'longitude': longitude,
        'camera_id': camera_id,
    }
    try:
        # The 'json' parameter automatically sets the Content-Type header
        async with session.post(API_ENDPOINT, json=payload, timeout=10) as response:
            if response.status >= 400:
                print(f"Error sending data for {species}: {response.status} {await response.text()}")
            else:
                print(f"Successfully sent data for {species}: {count}")
    except Exception as e:
        print(f"An error occurred while sending data for {species}: {e}")

# --- NEW: Thread to run the asyncio event loop ---
def run_asyncio_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

# --- helpers ---
def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a; bx1, by1, bx2, by2 = b
    inter_x1, inter_y1 = max(ax1, bx1), max(ay1, by1)
    inter_x2, inter_y2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, inter_x2 - inter_x1), max(0, inter_y2 - inter_y1)
    inter = iw * ih
    if inter == 0: return 0.0
    area_a = (ax2 - ax1) * (ay2 - ay1)
    area_b = (bx2 - bx1) * (by2 - by1)
    return inter / float(area_a + area_b - inter + 1e-6)

def centroid(box):
    x1, y1, x2, y2 = box
    return (0.5*(x1+x2), 0.5*(y1+y2))

# --- entity store ---
class Entity:
    _next_id = 0
    def __init__(self, box, frame_idx, class_name, track_id=None):
        self.eid = Entity._next_id; Entity._next_id += 1
        self.box = box; self.last_frame = frame_idx
        self.class_hist = Counter([class_name]); self.canonical = class_name
        self.track_ids = set()
        if track_id is not None: self.track_ids.add(int(track_id))
    def update(self, box, frame_idx, class_name, track_id=None):
        self.box = box; self.last_frame = frame_idx
        self.class_hist[class_name] += 1
        self.canonical = self.class_hist.most_common(1)[0][0]
        if track_id is not None: self.track_ids.add(int(track_id))

# --- main ---
# --- NEW: Setup for asyncio in a separate thread ---
async_loop = asyncio.new_event_loop()
async_thread = threading.Thread(target=run_asyncio_loop, args=(async_loop,), daemon=True)
async_thread.start()
# We create one session to be reused for all requests for efficiency
session_future = asyncio.run_coroutine_threadsafe(aiohttp.ClientSession().__aenter__(), async_loop)
session = session_future.result()


model = YOLO(WEIGHTS)
cap = cv2.VideoCapture(VIDEO)
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
diag = (w**2 + h**2) ** 0.5
max_age_frames = int(MAX_AGE_SEC * fps)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("output_video.mp4", fourcc, fps, (w, h))
entities = []
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    frame_idx += 1

    results = model.track(frame, imgsz=IMG_SIZE, conf=CONF_THRES, tracker=TRACKER, persist=True, verbose=False)
    annotated = results[0].plot()
    boxes = results[0].boxes
    if boxes is not None and len(boxes) > 0:
        xyxy = boxes.xyxy.cpu().numpy()
        clsi = boxes.cls.cpu().numpy().astype(int)
        names = [model.names[i] for i in clsi]
        tids = boxes.id.cpu().numpy().astype(int).tolist() if boxes.id is not None else [None]*len(xyxy)
        for box, name, tid in zip(xyxy, names, tids):
            if CLASS_WHITELIST and name not in CLASS_WHITELIST: continue
            best_e, best_score = None, -1
            cx, cy = centroid(box)
            for e in entities:
                if frame_idx - e.last_frame > max_age_frames: continue
                iou = iou_xyxy(box, e.box)
                ex, ey = centroid(e.box)
                dist_ok = ( ((cx-ex)**2 + (cy-ey)**2)**0.5 ) < (DIST_FRAC * diag )
                score = iou + (0.5 if dist_ok else 0.0)
                if score > best_score and (iou >= IOU_MERGE or dist_ok):
                    best_e, best_score = e, score
            if best_e is None:
                entities.append(Entity(box, frame_idx, name, tid))
            else:
                best_e.update(box, frame_idx, name, tid)

    entities = [e for e in entities if frame_idx - e.last_frame <= 5*max_age_frames]
    counts = Counter(e.canonical for e in entities if not CLASS_WHITELIST or e.canonical in CLASS_WHITELIST)
    total_unique = sum(counts.values())

    # --- NEW: Trigger data sending ---
    if frame_idx % SEND_INTERVAL_FRAMES == 0 and total_unique > 0:
        print(f"\n--- Frame {frame_idx}: Sending data ---")
        # --- TO DO: Fill these with actual data ---
        behaviour = "grazing"
        latitude = 19.0760
        longitude = 72.8777
        camera_id = "CAM-001"
        
        for species, count in counts.items():
            # Create a coroutine object for the API call
            coro = send_data(session, species, count, behaviour, latitude, longitude, camera_id)
            # Submit it to the event loop running in the other thread
            asyncio.run_coroutine_threadsafe(coro, async_loop)

    # overlay
    y = 40
    cv2.putText(annotated, f"Total Unique Animals: {total_unique}", (20, y),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0,255,0), 4); y += 40
    for cls, c in counts.items():
        cv2.putText(annotated, f"{cls}: {c}", (20, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,255), 3); y += 32
    
    cv2.imshow("YOLO Video", annotated)
    out.write(annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

# --- NEW: Cleanup ---
print("Closing resources...")
# Close the aiohttp session
asyncio.run_coroutine_threadsafe(session.__aexit__(None, None, None), async_loop)
# Stop the event loop
async_loop.call_soon_threadsafe(async_loop.stop)
# Wait for the thread to finish
async_thread.join()

cap.release()
out.release()
cv2.destroyAllWindows()
final_counts = Counter(e.canonical for e in entities if not CLASS_WHITELIST or e.canonical in CLASS_WHITELIST)
print("Final unique animals:", dict(final_counts))

In [19]:
def sendData(species, count, behaviour, latitude, longitude, camera_id):
    url = 'http://127.0.0.1:8000/animals/'
    
    # Generate a timezone-aware timestamp in the correct ISO format
    # The 'Z' at the end is automatically handled by .isoformat() for UTC times
    timestamp_now = datetime.now(timezone.utc).isoformat()

    data = {
        'species': species,
        'count': count,
        'behaviour': behaviour,
        'latitude': latitude,
        'longitude': longitude,
        'camera_id': camera_id,
        'timestamp': timestamp_now # Use the generated timestamp
    }
    
    try:
        response = requests.post(url, json=data)
        response.raise_for_status() 
        
        print(f"Success! Status: {response.status_code}")
        print("Response JSON:", response.json())

    except requests.exceptions.RequestException as e:
        # This part can be improved to show the specific error from Django
        if e.response:
            print(f"Error from server: {e.response.status_code}")
            print("Details:", e.response.json())
        else:
            print(f"An error occurred: {e}")

In [20]:
sendData("Panthera_leo", 3, "Walking", 12.9716, 77.5946, "CAM-001")

Success! Status: 201
Response JSON: {'id': 3, 'species': 'Panthera_leo', 'count': 3, 'behaviour': 'Walking', 'latitude': '12.971600', 'longitude': '77.594600', 'camera_id': 'CAM-001', 'timestamp': '2025-09-10T17:35:36.208660Z'}
